In [ ]:
!pip install langchain-teddynote markdownify pymupdf

In [29]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("Jimin-Parser-Test")

LangSmith 추적을 시작합니다.
[프로젝트명]
Jimin-Parser-Test


In [30]:
import os
import pymupdf
from glob import glob
import json
import requests
from PIL import Image

In [31]:
SAMPlE = "data/test-file.pdf"

## Layout parsing algorithm

레이아웃에서 가져온 이미지를 벡터디비에 넣고, 이미지를 다시 우리한테 꺼내서 보여줘야할 줄 알아야함. layout parser는 이 영역의 이미지가 존재한다를 알려주는거지, 이 이미지를 따로 따서 저장하지는 않음. 

- table도 이미지 캡처떠서 멀티 모달에 넣어서 그거에 마크다운 변환해서 넣어주면 성능이 좋아질것 같음. 


### 1. PDF 파일 분할

#### 왜 분할 할까?
- 100페이지 이상의 PDF 파일을 한 번에 처리할 경우 Upstage layout parsing API 오류 발생
- 처리 효율성 및 속도 개선
- 단일 페이지 처리는 api 전체 처리 시간을 증가시킴 

#### 일케 해봅세
- PDF 파일을 적절한 크기의 세그먼트로 분할하여 처리
- 분할된 세그먼트를 순차적으로 API에 전송하여 처리 효율성 향상

In [32]:
filepath = SAMPlE
batch_size = 10

input_pdf = pymupdf.open(filepath)
num_pages = len(input_pdf)
print(f"총 페이지 수: {num_pages}")


ret_split_files= []

# PDF 분할
for start_page in range(0,num_pages,batch_size):
    end_page = min(start_page + batch_size, num_pages) -1
    print(f"분할: {start_page} ~ {end_page}")
    
    
    # 분할된 PDF를 저장
    input_file_basename = os.path.splitext(filepath)[0]
    output_file = f"{input_file_basename}_{start_page}_{end_page}.pdf"
    print(f"분할 PDF 저장: {output_file}")
    with pymupdf.open() as output_pdf:
        output_pdf.insert_pdf(input_pdf, from_page=start_page, to_page=end_page)
        output_pdf.save(output_file)
        ret_split_files.append(output_file)
        print(f"{output_file} ret에 추가 ")
        
input_pdf.close()

총 페이지 수: 16
분할: 0 ~ 9
분할 PDF 저장: data/test-file_0_9.pdf
data/test-file_0_9.pdf ret에 추가 
분할: 10 ~ 15
분할 PDF 저장: data/test-file_10_15.pdf
data/test-file_10_15.pdf ret에 추가 


In [33]:
print(ret_split_files)
split_files = ret_split_files

['data/test-file_0_9.pdf', 'data/test-file_10_15.pdf']


### Upstage Layout Analyer
[Layout Analyzer](https://www.content.upstage.ai/blog/business/introducing-layout-analysis)

[Layout Analyzer Quick Start](https://developers.upstage.ai/docs/apis/document-parse)

upstage layout analyer에 없는 별도의 클래스이다. 이 코드는 *테디* 님 코드를 참고하였다.


언제 씀?
- ocr 필요하면 -> true로 쓰기 

언제 필요함?
- pdf 텍스트가 긁어지면 x 
- 안긁어지면 true


In [34]:
class LayoutAnalyzer:
    def __init__(self, api_key):
        self.api_key = api_key

    def _upstage_layout_analysis(self, input_file):
        """
        레이아웃 분석 API 호출

        :param input_file: 분석할 PDF 파일 경로
        :param output_file: 분석 결과를 저장할 JSON 파일 경로
        """
        # API 요청 보내기
        response = requests.post(
            "https://api.upstage.ai/v1/document-ai/layout-analysis",
            headers={"Authorization": f"Bearer {self.api_key}"},
            data={"ocr": True},
            files={"document": open(input_file, "rb")},
        )

        # 응답 저장
        if response.status_code == 200:
            output_file = os.path.splitext(input_file)[0] + ".json"
            with open(output_file, "w") as f:
                json.dump(response.json(), f, ensure_ascii=False)
            return output_file
        else:
            raise ValueError(f"예상치 못한 상태 코드: {response.status_code}")

    def execute(self, input_file):
        return self._upstage_layout_analysis(input_file)

In [35]:
analyzer = LayoutAnalyzer(os.environ.get("UPSTAGE_API_KEY"))

analyzed_files = []

for file in split_files:
    analyzed_files.append(analyzer.execute(file))

In [36]:
analyzed_files

['data/test-file_0_9.json', 'data/test-file_10_15.json']